In [ ]:
path = "./data/tweet.js"

with open(path, "r") as f:
    raw_text = f.read()

In [ ]:
raw_text = raw_text.replace("window.YTD.tweet.part0 = ", "")

In [ ]:
import json
from pprint import pprint
from dateutil.parser import parse

data = json.loads(raw_text)

In [ ]:
for tweet in data:
    tweet = tweet['tweet']
    id = tweet["id_str"]
    date = str(parse(tweet['created_at'])).split("+")[0]
    text = tweet["full_text"]
    pprint(tweet)
    print()
    print()

In [ ]:
main_dict = {}
date_list = []

for tweet in data:
    tweet = tweet['tweet']
    id = tweet["id_str"]
    date = str(parse(tweet['created_at'])).split("+")[0]
    text = tweet["full_text"]
    if "in_reply_to_status_id_str" not in tweet:
        a_tweet_dict = {
            "date": date,
            "text": text,
        }
        main_dict.update({id: a_tweet_dict})
        date_list.append(date)

for tweet in data:
    tweet = tweet['tweet']
    id = tweet["id_str"]
    date = str(parse(tweet['created_at'])).split("+")[0]
    text = tweet["full_text"]
    if "in_reply_to_status_id_str" in tweet:
        reply_id = tweet["in_reply_to_status_id_str"]
        if reply_id in main_dict.keys():
            main_dict[reply_id]['text'] += "\n\n\n\n" + text
        else:
            pass
        
main_list = list(main_dict.values())

# take second element for sort
def return_date(elem):
    return elem["date"]

# sort list with key
main_list.sort(key=return_date)

pprint(main_list[:10])

In [ ]:
for oneday in main_list[-5:]:
    date = oneday["date"]
    text = oneday["text"]
    print(text)
    print("\n\n----------------\n\n")

In [ ]:
!rm tweets.db

In [ ]:
import os
import sqlite3

class Self():
    _root_path = "."
    pass

self = Self()
self.SQL_DATA_FILE = os.path.join(self._root_path, "tweets.db")
if not os.path.exists(self.SQL_DATA_FILE):
    print(f"SQL doesn't exist: {self.SQL_DATA_FILE}")
    print(f"We'll create a new one!")
self._sql_conn = sqlite3.connect(self.SQL_DATA_FILE, check_same_thread=False)
self._sql_cursor = self._sql_conn.cursor()
self._sql_cursor.execute('''CREATE TABLE IF NOT EXISTS thoughts
            (date TEXT, type TEXT, text TEXT, images TEXT)''')

for oneday in main_list:
    date = oneday["date"]
    text = oneday["text"]
    day = (date, "twitter", text, json.dumps([]))
    self._sql_cursor.execute("INSERT INTO thoughts VALUES (?,?,?,?)", day)
    self._sql_conn.commit()